In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, SimpleRNN
from keras.callbacks import ModelCheckpoint
from random import randint
from matplotlib import pyplot as pl

Using TensorFlow backend.


In [2]:
with open("sonnets.txt") as corpus_file:
    corpus = corpus_file.read()
    corpus_length = len(corpus)
print("Loaded a corpus of {0} characters".format(corpus_length))

Loaded a corpus of 94652 characters


In [3]:
# Get a unique identifier for each char in the corpus, 
# then make some dicts to ease encoding and decoding
chars = sorted(list(set(corpus)))
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}
print("Our corpus contains {0} unique characters.".format(num_chars))

Our corpus contains 62 unique characters.


In [4]:
# chop up our data into X and y, slice into roughly 
# (num_chars / skip) overlapping 'sentences' of length 
# sentence_length, and encode the chars
sentence_length = 20
skip = 1
X_data = []
y_data = []
for i in range (0, len(corpus) - sentence_length, skip):
    sentence = corpus[i:i + sentence_length]
    next_char = corpus[i + sentence_length]
    X_data.append([encoding[char] for char in sentence])
    y_data.append(encoding[next_char])

num_sentences = len(X_data)
print("Sliced our corpus into {0} sentences of length {1}"
      .format(num_sentences, sentence_length))

Sliced our corpus into 94632 sentences of length 20


In [5]:
print(X_data[1])

[17, 52, 49, 47, 1, 40, 35, 43, 52, 39, 53, 54, 1, 37, 52, 39, 35, 54, 55, 52]


In [6]:
print([decoding[idx] for idx in X_data[1]])
print(decoding[y_data[1]])

['F', 'r', 'o', 'm', ' ', 'f', 'a', 'i', 'r', 'e', 's', 't', ' ', 'c', 'r', 'e', 'a', 't', 'u', 'r']
e


In [7]:
# Vectorize our data and labels. We want everything in one-hot.
print("Vectorizing X and y...")
X = np.zeros((num_sentences, sentence_length, num_chars), dtype=np.bool)
y = np.zeros((num_sentences, num_chars), dtype=np.bool)
for i, sentence in enumerate(X_data):
    for t, encoded_char in enumerate(sentence):
        X[i, t, encoded_char] = 1
    y[i, y_data[i]] = 1

Vectorizing X and y...


In [8]:
# Double check our vectorized data before we sink hours into fitting a model
print("Sanity check y. Dimension: {0} # Sentences: {1} Characters in corpus: {2}"
      .format(y.shape, num_sentences, len(chars)))
print("Sanity check X. Dimension: {0} Sentence length: {1}"
      .format(X.shape, sentence_length))

Sanity check y. Dimension: (94632, 62) # Sentences: 94632 Characters in corpus: 62
Sanity check X. Dimension: (94632, 20, 62) Sentence length: 20


In [9]:
# Define our model
model = Sequential()
model.add(SimpleRNN(256, input_shape=(sentence_length, num_chars), 
                    return_sequences=False))
model.add(Dense(num_chars))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 256)               81664     
_________________________________________________________________
dense_1 (Dense)              (None, 62)                15934     
_________________________________________________________________
activation_1 (Activation)    (None, 62)                0         
Total params: 97,598
Trainable params: 97,598
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#training time
log = model.fit(X, y, epochs=50, batch_size=128)

Epoch 1/50
94632/94632 [==============================] - 43s 452us/step - loss: 2.5150 - acc: 0.3061
Epoch 2/50
94632/94632 [==============================] - 38s 404us/step - loss: 2.1723 - acc: 0.3697
Epoch 3/50
20096/94632 [=====>........................] - ETA: 42s - loss: 2.1174 - acc: 0.3778 ETA: 42s - lo

In [ ]:
pl.plot(log.history['loss'], label='Training')
pl.legend()
pl.grid()

In [ ]:
def make_seed(seed_phrase=""):
        if seed_phrase:
            phrase_length = len(seed_phrase)
            pattern = ""
            for i in range (0, sentence_length):
                pattern += seed_phrase[i % phrase_length]
        else:
            seed = randint(0, corpus_length - sentence_length)
            pattern = corpus[seed:seed + sentence_length]

        return pattern

In [ ]:
seed_pattern = make_seed("In the early morning, the flower is shining")
print(seed_pattern)

In [ ]:
X = np.zeros((1, sentence_length, num_chars), dtype=np.bool)
for i, character in enumerate(seed_pattern):
    X[0, i, encoding[character]] = 1

generated_text = ""
for i in range(500):
    output_prob = model.predict(X, verbose=0)[0]
    # alternative is to use argmax: prediction = np.argmax(output_prob)
    prediction = np.random.choice(num_chars, p = output_prob )
    generated_text += decoding[prediction]
    activations = np.zeros((1, 1, num_chars), dtype=np.bool)
    activations[0, 0, prediction] = 1
    #now remove first char and glue the predicted one
    X = np.concatenate((X[:, 1:, :], activations), axis=1)
print(generated_text)